# Multiple Linear Regression 
With more than one features x>=2

For any prediction there could be more than one deciding factors. For example, Price of the houses not only depend on the area of the house, But also it depends on multiple factors such as, Number of bedrooms, Locality, age of the house and so on. 

Our task is to design the model which predicts the price of the house appropriately. We use most widely used algorithm, Gradient descent.

## 2 Problem Statement

You will use the motivating example of housing price prediction. The training dataset contains three examples with four features (size, bedrooms, floors and, age) shown in the table below.  Note that, unlike the earlier labs, size is in sqft rather than 1000 sqft. This causes an issue, which you will solve in the next lab!

| Size (sqft) | Number of Bedrooms  | Number of floors | Age of  Home | Price (1000s dollars)  |   
| ----------------| ------------------- |----------------- |--------------|-------------- |  
| 2104            | 5                   | 1                | 45           | 460           |  
| 1416            | 3                   | 2                | 40           | 232           |  
| 852             | 2                   | 1                | 35           | 178           |  

You will build a linear regression model using these values so you can then predict the price for other houses. For example, a house with 1200 sqft, 3 bedrooms, 1 floor, 40 years old.  


In [98]:
import copy, math
import numpy as np
import matplotlib.pyplot as plt
x_train = np.array([[2104,5,1,45],[1416,3,2,40],[852,2,1,35]])
y_train = np.array([460,232,178])
x_train,y_train


(array([[2104,    5,    1,   45],
        [1416,    3,    2,   40],
        [ 852,    2,    1,   35]]),
 array([460, 232, 178]))

In [10]:
print(f"x_train.shape is : {x_train.shape}")
print(f"y_train.shape is : {y_train.shape}")

x_train.shape is : (3, 4)
y_train.shape is : (3,)


x_train is a multi dimentional vector of size (m,n)
$$\mathbf{X} = 
\begin{pmatrix}
 x^{(0)}_0 & x^{(0)}_1 & \cdots & x^{(0)}_{n-1} \\ 
 x^{(1)}_0 & x^{(1)}_1 & \cdots & x^{(1)}_{n-1} \\
 \cdots \\
 x^{(m-1)}_0 & x^{(m-1)}_1 & \cdots & x^{(m-1)}_{n-1} 
\end{pmatrix}
$$

w (parameter) is vector with single dimension. Because, considering the example each feature will have a single parameter w
$$\mathbf{w} = \begin{pmatrix}
w_0 \\ 
w_1 \\
\cdots\\
w_{n-1}
\end{pmatrix}
$$
* $b$ is a scalar parameter.  

In [21]:
b_init = 785.1811367994083
w_init = np.array([ 0.39133535, 18.75376741, -53.36032453, -26.42131618])

### find prediction for 1-d single
$$ f_{\mathbf{w},b}(\mathbf{x}) =  w_0x_0 + w_1x_1 +... + w_{n-1}x_{n-1} + b \tag{1}$$
or in vector notation:
$$ f_{\mathbf{w},b}(\mathbf{x}) = \mathbf{w} \cdot \mathbf{x} + b  \tag{2} $$ 
where $\cdot$ is a vector `dot product`

In [50]:
def find_prediction(x,w,b):
    n = x.shape[0]
    p = pi = 0 # prediction scalar
    for i in range(n):
       pi = pi + w[i]*x[i]
    p = pi + b
    return p
def find_prediction_dot(x,w,b):
    p = np.dot(w,x) + b
    return p


now let's predict for with a single row of features

In [35]:
x = x_train[0,:]
print(f" prediction with conventional for loop : {find_prediction(x,w_init,b_init)}")
print(f" prediction with dot product : {fin_prediction_dot(x,w_init,b_init)}")

 prediction with conventional for loop : 459.9999976194083
 prediction with dot product : 459.9999976194083


#### Now let's compute cost function
The equation for the cost function with multiple variables $J(\mathbf{w},b)$ is:
$$J(\mathbf{w},b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2 \tag{3}$$ 
where:
$$ f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)} + b  \tag{4} $$ 


In contrast to previous labs, $\mathbf{w}$ and $\mathbf{x}^{(i)}$ are vectors rather than scalars supporting multiple features.


In [108]:
def compute_cost_multi(x,w,b,y,prediction_function):
    #x is a vector of multi dimension (m,n) of m rows and n columns -- m example(samples) and n features
    #w is a vector of single dimension (,n) -- model parameter vector
    #b is a scalar value - base parameter scalar
    #y actual value vector (m,)
    #c_cost is cost  scalar
    c_cost = 0
    m = x.shape[0]
    for i in range(m):
        p = prediction_function(x[i],w,b)
        c_cost = c_cost + ((p - y[i])**2)
    c_cost = c_cost / (2 * m)                      #scalar  
    return c_cost
def compute_cost(x,y,w,b):
    m =x.shape[0]
    cost = 0
    for i in range(m):
        cost = cost + (np.dot(x[i],w) - y[i])**2
    cost = cost/2*m
    return cost

In [67]:
compute_cost_multi(x_train,w_init,b_init,y_train,find_prediction_dot)

1.5578904045996674e-12

## Gradient descent with multiple variables
Above we calculated the cost for a w = [$w_{0}$ , $w_{1}$ , $w_{3}$ ... $w_{n-1}$]

Idea of gradient descent is to arrive at miniumum value of cost for w and b
final output we are looking to arrive at is $w_{m,n}$ vector with m samples and n features. and b scalar value.
initialise w_init = [$w_{0}$ , $w_{1}$ , $w_{3}$ ... $w_{n-1}$] and b_init = 0
and with learning factor $\alpha$


$$\begin{align*} \text{repeat}&\text{ until convergence:} \; \lbrace \newline\;
& w_j = w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{5}  \; & \text{for j = 0..n-1}\newline
&b\ \ = b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b}  \newline \rbrace
\end{align*}$$

where, n is the number of features, parameters $w_j$,  $b$, are updated simultaneously and where  

$$
\begin{align}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \tag{6}  \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{7}
\end{align}
$$
* m is the number of training examples in the data set

    
*  $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ is the model's prediction, while $y^{(i)}$ is the target value

In [102]:
def compute_gradients_multi_var(x,y,w,b):
    m,n = x_train.shape
    err = 0
    dj_w = np.zeros((n,))
    dj_b = 0
    for i in range(m):
        err = (np.dot(x[i],w) + b) - y[i]
        for j in range(n):
            dj_w[j] = dj_w[j] + err * x[i,j]
        dj_b = dj_b + err
    return dj_w/m , dj_b/m


In [103]:
tmp_dj_dw, tmp_dj_db  = compute_gradients_multi_var(x_train, y_train, w_init, b_init)
print(f"gradients are dj_w : {tmp_dj_dw} and dj_b = {tmp_dj_db}")

gradients are dj_w : [-2.72623574e-03 -6.27197255e-06 -2.21745574e-06 -6.92403377e-05] and dj_b = -1.6739251122999121e-06


#### Gradient descent

In [154]:
    
def compute_gradient_descent(X,y,w_in,b_in,gradient_funtion,cost_function,alpha,iterations):
    j_Hist = [] # the list will be of length of number of iterations carried
    initial_w = np.zeros_like(w_init)
    w = copy.deepcopy(initial_w)
    b = b_in
    for i in range(iterations):
        #print(f"this is {i}th loop")
        grdadient = gradient_funtion(X,y,w,b)
        w = w - (alpha * (grdadient)[0])
        b = b - (alpha * (grdadient)[1])
        if i < 10000:
           j = cost_function(X,y,w,b)
           j_Hist.append(j)
        if i % 10 == 0:
            #print(j_Hist)
            pass
    return w,b,j_Hist
    

In [160]:
b_init = 0.
iterations = 1000
alpha = 5.0e-7
w_final, b_final, j_final = compute_gradient_descent(x_train,y_train,w_init,b_init,compute_gradients_multi_var,compute_cost,alpha,iterations)
print(f"final w found by the model is : {w_final} final b found by the model is {b_final}")
w_init

final w found by the model is : [ 0.20396569  0.00374919 -0.0112487  -0.0658614 ] final b found by the model is -0.0022354075309325345


array([  0.39133535,  18.75376741, -53.36032453, -26.42131618])

In [146]:
10 % 10 == 0

True

In [119]:
y_train

array([460, 232, 178])

In [120]:
w_init

array([  0.39133535,  18.75376741, -53.36032453, -26.42131618])

In [122]:
b_init

785.1811367994083

In [157]:
a = 5.0e-7

4.9999999999999996e-06